In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
torch.cuda.is_available()

False

In [3]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

In [4]:
batch_size = 100
num_epochs = 10
n_iters = int((len(train_dataset) / batch_size ) * num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [5]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        return out

In [6]:
model = FeedforwardNeuralNetModel(28*28, 100, 10)

if torch.cuda.is_available():
    model.cuda()

criterion = nn.CrossEntropyLoss()

learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        if torch.cuda.is_available():
            images = Variable(images.view(-1, 28*28).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, 28*28))
            labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    # Calculate Accuracy per epoch       
    correct = 0
    total = 0
    for images, labels in test_loader:
        if torch.cuda.is_available():
            images = Variable(images.view(-1, 28*28).cuda())
        else:
            images = Variable(images.view(-1, 28*28))
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # Total correct predictions
        correct += (predicted.cpu() == labels.cpu()).sum()

    accuracy = 100 * correct / total

    # Print Loss
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.data[0], accuracy))


Epoch: 0. Loss: 0.2851710319519043. Accuracy: 91.55
Epoch: 1. Loss: 0.16056017577648163. Accuracy: 94.63
Epoch: 2. Loss: 0.22978651523590088. Accuracy: 95.95
Epoch: 3. Loss: 0.09497270733118057. Accuracy: 96.47
Epoch: 4. Loss: 0.09756559133529663. Accuracy: 96.67
Epoch: 5. Loss: 0.051919545978307724. Accuracy: 97.09
